In [ ]:
import torch
import numpy as np

In [7]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [8]:
criterion = torch.nn.BCEWithLogitsLoss()

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

class HairCNN(nn.Module):
    def __init__(self):
        super(HairCNN, self).__init__()
        # Capa convolucional
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=0
        )
        # Max-pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Después de conv+pool: 32 x 99 x 99
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)  # (batch, 32*99*99)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)          # logits (sin sigmoid)
        return x


model = HairCNN().to(device)
print(model)




Device: cpu
HairCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [14]:
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

Total parameters: 20073473


In [15]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


In [16]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = datasets.ImageFolder('data/train', transform=train_transforms)
validation_dataset = datasets.ImageFolder('data/test', transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)


In [17]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)


In [18]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)  # (batch, 1)

        optimizer.zero_grad()
        outputs = model(images)             # logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        preds = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (preds == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # validación
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (preds == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")


Epoch 1/10, Loss: 0.6586, Acc: 0.6125, Val Loss: 0.6080, Val Acc: 0.6567
Epoch 2/10, Loss: 0.5467, Acc: 0.7188, Val Loss: 0.5846, Val Acc: 0.6716
Epoch 3/10, Loss: 0.5127, Acc: 0.7475, Val Loss: 0.6183, Val Acc: 0.6617
Epoch 4/10, Loss: 0.4621, Acc: 0.7762, Val Loss: 0.6244, Val Acc: 0.6915
Epoch 5/10, Loss: 0.4185, Acc: 0.8075, Val Loss: 0.6095, Val Acc: 0.6816
Epoch 6/10, Loss: 0.3805, Acc: 0.8313, Val Loss: 0.7474, Val Acc: 0.6468
Epoch 7/10, Loss: 0.2969, Acc: 0.8800, Val Loss: 0.6547, Val Acc: 0.7264
Epoch 8/10, Loss: 0.2143, Acc: 0.9287, Val Loss: 0.7545, Val Acc: 0.6866
Epoch 9/10, Loss: 0.2472, Acc: 0.8900, Val Loss: 1.2412, Val Acc: 0.6269
Epoch 10/10, Loss: 0.1975, Acc: 0.9287, Val Loss: 0.6776, Val Acc: 0.7363


In [19]:
train_accuracies = np.array(history['acc'])
median_train_acc = np.median(train_accuracies)
median_train_acc

np.float64(0.819375)

In [20]:
train_losses = np.array(history['loss'])
std_train_loss = np.std(train_losses)   # por defecto, población (ddof=0)
std_train_loss

np.float64(0.146537893669849)

In [21]:
std_train_loss_sample = np.std(train_losses, ddof=1)

In [22]:
augmented_train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [23]:
train_dataset_aug = datasets.ImageFolder('data/train', transform=augmented_train_transforms)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)


In [24]:
num_extra_epochs = 10

val_losses_aug_epochs = []

for epoch in range(num_extra_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # ENTRENAMIENTO CON AUGMENTATION
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (preds == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train

    # VALIDACIÓN (test) SIN AUGMENTATIONS
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (preds == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val

    val_losses_aug_epochs.append(val_epoch_loss)

    print(f"[AUG] Epoch {epoch+1}/{num_extra_epochs}, "
          f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")


[AUG] Epoch 1/10, Train Loss: 0.5986, Train Acc: 0.7087, Val Loss: 0.6460, Val Acc: 0.6866
[AUG] Epoch 2/10, Train Loss: 0.5658, Train Acc: 0.7000, Val Loss: 0.5702, Val Acc: 0.6915
[AUG] Epoch 3/10, Train Loss: 0.5024, Train Acc: 0.7400, Val Loss: 0.5707, Val Acc: 0.6915
[AUG] Epoch 4/10, Train Loss: 0.5204, Train Acc: 0.7312, Val Loss: 0.5199, Val Acc: 0.7413
[AUG] Epoch 5/10, Train Loss: 0.4840, Train Acc: 0.7400, Val Loss: 0.5517, Val Acc: 0.7015
[AUG] Epoch 6/10, Train Loss: 0.4755, Train Acc: 0.7650, Val Loss: 0.4851, Val Acc: 0.7811
[AUG] Epoch 7/10, Train Loss: 0.4707, Train Acc: 0.7662, Val Loss: 0.4987, Val Acc: 0.7512
[AUG] Epoch 8/10, Train Loss: 0.4311, Train Acc: 0.7875, Val Loss: 0.4980, Val Acc: 0.7562
[AUG] Epoch 9/10, Train Loss: 0.4892, Train Acc: 0.7625, Val Loss: 0.5177, Val Acc: 0.7313
[AUG] Epoch 10/10, Train Loss: 0.4396, Train Acc: 0.7900, Val Loss: 0.5064, Val Acc: 0.7463


In [ ]:
val_losses_aug_epochs = np.array(val_losses_aug_epochs)
mean_val_loss_aug = np.mean(val_losses_aug_epochs)
mean_val_loss_aug


np.float64(0.536462057111275)

In [32]:
val_accs_aug_epochs = []

for epoch in range(num_extra_epochs):


    val_losses_aug_epochs.append(val_epoch_loss)
    val_accs_aug_epochs.append(val_epoch_acc)



AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [31]:
val_accs_aug_epochs

[]

In [30]:
last_5_accs = val_accs_aug_epochs[5:]   # índices 5,6,7,8,9
mean_last_5_acc = np.mean(last_5_accs)
mean_last_5_acc



np.float64(nan)